# ANALYZE EARNINGS CALLS AND PRESENT A STORY LIKE A PRO BUSINESS ANALYST 


## USING CEREBRAS + OPENROUTER + QWEN3-32 

### Saurabh Daptardar

## STEP 0 : IMPORTS AND CLIENT FOR MODEL ENDPOINT

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
from pprint import pprint
from IPython.display import display, Markdown

# Load environment variables from .env file
load_dotenv()

# Ensure OPENROUTER_API_KEY is set in the environment
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY environment variable is not set.")


# Initialize OpenAI client with OpenRouter API key
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)


## STEP 0 : Smoke Test for basic functionality

In [2]:

# Create a chat completion using the Qwen-3 32B model from Cerebras
completion = client.chat.completions.create(
#   extra_headers={
#     "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
#     "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
#   },
  extra_body={
    "provider": {
      "only": ["Cerebras"]
    }
  },
  model="qwen/qwen3-32b",
  messages=[
    {
      "role": "user",
      "content": "Explain the meaning of life in four sentences."
    }
  ]
)
print(completion.choices[0].message.content)

1. The meaning of life is a subjective and multifaceted concept, shaped by personal values, cultural contexts, and the pursuit of purpose or fulfillment.  
2. Philosophically, it may be framed as a search for existence's inherent logic, with existentialist thought asserting that individuals create their own meaning through choices and actions.  
3. Biologically, life’s "purpose" could be interpreted as survival, adaptation, and the propagation of life through evolutionary processes.  
4. Across spiritual traditions, the meaning often centers on connection—whether to a higher power, to others through compassion, or to the universe as part of an interconnected whole.


## STEP 0 : Smoke Test for reasoning

In [3]:
completion = client.chat.completions.create(
#   extra_headers={
#     "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
#     "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
#   },
  extra_body={
    "provider": {
      "only": ["Cerebras"]
    },
    "chat_template_kwargs": {"enable_thinking": True},  # Enable reasoning steps
    "separate_reasoning": True  # Return reasoning steps separately
  },
  model="qwen/qwen3-32b",
  messages=[
    {
      "role": "user",
      "content": "Explain the meaning of life in four sentences."
    }
  ]
)

# Print the reasoning steps and final answer
pprint("Reasoning Steps:")
pprint(completion.choices[0].message.reasoning)
pprint("\nFinal Answer:")
pprint(completion.choices[0].message.content)

## Uncomment the following line to see the full message structure
# pprint(completion.choices[0].message.to_dict())

'Reasoning Steps:'
('\n'
 'Okay, the user wants an explanation of the meaning of life in four '
 'sentences. Let me start by considering different philosophical perspectives. '
 'Existentialism suggests creating our own meaning, while religious views '
 'often point to serving a higher purpose. I should mention both to cover a '
 'range.\n'
 '\n'
 'Next, the idea that meaning is personal and varies between individuals is '
 "important. It's not one-size-fits-all. Then, including elements like "
 'connection, growth, and contribution could show a holistic approach. Maybe '
 "touch on how questioning life's meaning is part of the human experience and "
 'how answers evolve over time.\n'
 '\n'
 'Wait, the user might be looking for something concise but comprehensive. Let '
 'me make sure each sentence addresses a different angle. First, the '
 'philosophical debate. Second, the subjective nature. Third, specific '
 'components of meaning. Fourth, the dynamic process of seeking. Need to av

## STEP 0 : DATASET [Earnings Calls transcripts from Alphabet (Google) from Q1 2022 to Q1 2025]

In [4]:
years = list(range(2022, 2025))
quarters = ["q1", "q2", "q3", "q4"]
files = [ f"../data/{year}-{quarter}-earnings-call.txt" for year in years for quarter in quarters ]
additional_files = ["../data/2025-q1-earnings-call.txt"]
files.extend(additional_files)
# Print the list of files
pprint(files)

['../data/2022-q1-earnings-call.txt',
 '../data/2022-q2-earnings-call.txt',
 '../data/2022-q3-earnings-call.txt',
 '../data/2022-q4-earnings-call.txt',
 '../data/2023-q1-earnings-call.txt',
 '../data/2023-q2-earnings-call.txt',
 '../data/2023-q3-earnings-call.txt',
 '../data/2023-q4-earnings-call.txt',
 '../data/2024-q1-earnings-call.txt',
 '../data/2024-q2-earnings-call.txt',
 '../data/2024-q3-earnings-call.txt',
 '../data/2024-q4-earnings-call.txt',
 '../data/2025-q1-earnings-call.txt']


## STEP 1 :  Ask the LLM for key points the analyst should look for in the earnings call transcript

In [5]:
# Ennlist the key points that a business analyst should focus on when reading these earnings calls
completion = client.chat.completions.create(
#   extra_headers={
#     "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
#     "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
#   },
  extra_body={
    "provider": {
      "only": ["Cerebras"]
    },
    "chat_template_kwargs": {"enable_thinking": True},  # Enable reasoning steps
    "separate_reasoning": True  # Return reasoning steps separately
  },
  model="qwen/qwen3-32b",
  messages=[
    {
      "role": "user",
      "content": """What are the key points a business analyst should focus on when reading earnings calls?
      Convert the output to a JSON schema definition"""
    }
  ]
)
# Print the key points for business analysts
# pprint("Key Points for Business Analysts:")
# pprint(completion.choices[0].message.content)
key_points = completion.choices[0].message.content
# Save the key points to a file
with open("../outputs/key_points_business_analysts.json", "w") as f:
    f.write(key_points)

## STEP 2 : Considering  the key points in step 1, analyze the earnings calls

In [6]:
# Considering the key points, process each earnings call file and extract the key points in a structured JSON format
for file in files:
    with open(file, "r") as f:
        earnings_call_content = f.read()
    # Create a chat completion to extract key points from the earnings call content
    completion = client.chat.completions.create(
#       extra_headers={
#         "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
#         "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
#       },
        extra_body={
            "provider": {
                "only": ["Cerebras"]
            },
            "chat_template_kwargs": {"enable_thinking": True},  # Enable reasoning steps
            "separate_reasoning": True  # Return reasoning steps separately
        },
        model="qwen/qwen3-32b",
        messages=[
            {
                "role": "user",
                "content": f"""Extract the key points from the earnings call transcript considering the following key points for business analysts:
                The output should be in JSON format.
                The output should be strictly adhere to the prescribed key points in JSON schema definition.
                The extracted key points should be concise and relevant to business analysts.
                \n\n## KEY POINTS FOR BUSINESS ANALYSTS (JSON SCHEMA DEFINITION)
                \n\n{key_points}
                \n\n## EARNINGS CALL CONTENT 
                \n\n{earnings_call_content}"""
            }
        ]
    )
    # Print the extracted key points
    # pprint(f"Key Points from {file}:")
    # pprint(completion.choices[0].message.content)
    # Save the key points to a file
    output_file = f"../outputs/key_points_{os.path.basename(file).replace('.txt', '.json')}"
    print(f"Saving key points to {output_file}")
    output = completion.choices[0].message.content
    output = output.replace("```json", "").replace("```", "").strip()  # Clean up the output
    with open(output_file, "w") as f:
        f.write(output)



Saving key points to ../outputs/key_points_2022-q1-earnings-call.json
Saving key points to ../outputs/key_points_2022-q2-earnings-call.json
Saving key points to ../outputs/key_points_2022-q3-earnings-call.json
Saving key points to ../outputs/key_points_2022-q4-earnings-call.json
Saving key points to ../outputs/key_points_2023-q1-earnings-call.json
Saving key points to ../outputs/key_points_2023-q2-earnings-call.json
Saving key points to ../outputs/key_points_2023-q3-earnings-call.json
Saving key points to ../outputs/key_points_2023-q4-earnings-call.json
Saving key points to ../outputs/key_points_2024-q1-earnings-call.json
Saving key points to ../outputs/key_points_2024-q2-earnings-call.json
Saving key points to ../outputs/key_points_2024-q3-earnings-call.json
Saving key points to ../outputs/key_points_2024-q4-earnings-call.json
Saving key points to ../outputs/key_points_2025-q1-earnings-call.json


## STEP 3 : Combine the keypoints into a story (timeline)

In [7]:
# Iterate the key points json files and load them into a single prompt string
key_points_prompt = ""
for file in files:
    output_file = f"../outputs/key_points_{os.path.basename(file).replace('.txt', '.json')}"
    with open(output_file, "r") as f:
        key_points_content = f.read()
    key_points_prompt += f"\n\n## KEY POINTS FROM {os.path.basename(file)}\n{key_points_content}"

    pprint(f"Loaded key points from {output_file}")


# pprint(key_points_prompt)

'Loaded key points from ../outputs/key_points_2022-q1-earnings-call.json'
'Loaded key points from ../outputs/key_points_2022-q2-earnings-call.json'
'Loaded key points from ../outputs/key_points_2022-q3-earnings-call.json'
'Loaded key points from ../outputs/key_points_2022-q4-earnings-call.json'
'Loaded key points from ../outputs/key_points_2023-q1-earnings-call.json'
'Loaded key points from ../outputs/key_points_2023-q2-earnings-call.json'
'Loaded key points from ../outputs/key_points_2023-q3-earnings-call.json'
'Loaded key points from ../outputs/key_points_2023-q4-earnings-call.json'
'Loaded key points from ../outputs/key_points_2024-q1-earnings-call.json'
'Loaded key points from ../outputs/key_points_2024-q2-earnings-call.json'
'Loaded key points from ../outputs/key_points_2024-q3-earnings-call.json'
'Loaded key points from ../outputs/key_points_2024-q4-earnings-call.json'
'Loaded key points from ../outputs/key_points_2025-q1-earnings-call.json'


In [8]:
# Considering the JSON schema definition and the key points extracted from the earnings calls, 
# create story for the business analyst to present to the management team
# spanning over all the quarters

completion = client.chat.completions.create(
#   extra_headers={
#     "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
#     "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
#   },
  extra_body={
    "provider": {
      "only": ["Cerebras"]
    },
    "chat_template_kwargs": {"enable_thinking": True},  # Enable reasoning steps
    "separate_reasoning": True  # Return reasoning steps separately 
  },
  model="qwen/qwen3-32b",
  messages=[
    {
      "role": "user",
      "content": f"""Based on the following key points extracted from the earnings calls, create a comprehensive story for the business analyst to present to the management team.
      The story should span over all the quarters and should be concise and relevant to business analysts.
      \n\n## KEY POINTS (JSON SCHEMA DEFINITION)
      \n\n{key_points}
      \n\n## KEY POINTS FROM EARNINGS CALLS
      \n\n{key_points_prompt}
      \n\n## OUTPUT FORMAT
      \n\nThe output shouls be in markdown format"""
    }
  ]
)
    
# Print the story for the business analyst
story = completion.choices[0].message.content
display(Markdown(story))

# Save the story to a file
with open("../outputs/story_business_analyst.md", "w") as f:
    f.write(story)

### Comprehensive Business Narrative: 2022–2025 Earnings Highlights  
*(For Internal Management Review by Business Analysts)*  

---

#### **2022: Laying the Foundation for AI-Driven Growth**  
- **Financial Performance**: Revenue grew steadily but faced headwinds from FX volatility (-3% Q1 2022) and Russia-Ukraine war impacts. Quarterly revenue reached $69.1B by Q3 2022, with Cloud revenue hitting $6.3B (36% YoY growth).  
- **Strategic Priorities**: Accelerated Cloud and AI investments, including the **Mandiant acquisition** (cybersecurity expansion) and **YouTube Shorts monetization** (30B+ daily views globally). Hybrid work solutions (Google Meet, Workspace) became central.  
- **Challenges**: FX headwinds, regulatory risks (EU Digital Markets Act), and competition in short-form video (vs. TikTok).  
- **Capital Allocation**: $70B stock buyback authorization in Q1 2022, with $52B repurchased by Q2. Capex focused on U.S. data centers.  
- **Tone**: **Cautiously optimistic**, balancing long-term AI/cloud bets with macroeconomic uncertainties.  

---

#### **2023: AI Integration and Cost Discipline**  
- **Breakthroughs**:  
  - AI became a **core growth driver** (PaLM, LaMDA models) and **differentiator** (AI Hypercomputer, Gemini models).  
  - **Google Cloud** passed $9.2B quarterly revenue (26% YoY growth) with 60% of Fortune 1000 customers.  
  - **YouTube Shorts** achieved 2B+ monthly users, with monetization ramps starting to materialize (e.g., Shorts Fund expansion).  
- **Financial Shifts**: Operating expenses stabilized (flat Q4 2023 vs. prior year) as cost re-engineering reduced inefficiencies (12K workforce reductions).  
- **New Challenges**: Macro-economic slowdowns, Microsoft/AWS cloud competition, and rising AI infrastructure costs.  
- **Capital Returns**: $59B buybacks in 2022; a **$0.20/share dividend** launched in 2024.  

---

#### **2024: AI Infrastructure Scaling and Monetization**  
- **Product Leadership**:  
  - Released **Gemini 1.5 Pro** and **Trillium TPU** (5X performance boost), cementing AI infrastructure leadership.  
  - **Search Generative Experience (SGE)** and **AI Overviews** drove advertiser demand (35% higher conversions via Smart Bidding).  
  - **YouTube Premium** and **Google One** subscriptions passed **100M paid users**.  
- **Financial Milestones**:  
  - Cloud revenue exceeded **$10B/quarter** for the first time (29% YoY growth).  
  - Q4 2024 revenue reached **$96.5B**, with **net income of $26.5B** (up from $13.6B in Q4 2022).  
  - Capex surged to **$75B in 2025** (focused on AI servers and data centers).  
- **Partnerships**: Strategic alliances (e.g., Oralce, Hugging Face) and **Waymo-Uber AV expansion** created new revenue streams.  

---

#### **2025: AI Monetization and Operational Efficiency**  
- **Maturation of AI Strategy**:  
  - **Gemini 2.5**, AI Agents, and **AI Mode in Search** drove user engagement (2X longer queries for 18-24 age group).  
  - **Wiz acquisition** for multi-cloud security and **Hyundai partnership** for autonomous vehicles signaled diversification.  
- **Financial Resilience**:  
  - Q1 2025 revenue of **$90.2B** (12% YoY) with **33.9% operating margin**, and net income of $34.5B (46% YoY growth).  
  - **270M+ paid subscriptions** (YouTube/Google One) achieved via tiered monetization.  
- **Future Playbooks**:  
  - **2025–2026 roadmap** includes Project Astra (multimodal AI agents) and **$75B in 2025 AI infrastructure Capex**.  
  - **Operational efficiencies** reduced AI Overviews costs by **90%** and Waymo ride costs via automation.  

---

### **Key Trends and Strategic Insights**  
1. **AI as the Growth Engine**:  
   - AI infrastructure (TPUs, Vertex AI) and application tools (SGE, Gemini) became the **core of differentiation**, driving revenue in Cloud, Search, and YouTube.  
   - **Cloud growth accelerated** from 36% YoY in 2022 to 35% CAGR by 2024, supported by enterprise AI adoption.  

2. **Capital Efficiency**:  
   - Operating expenses stabilized (flat Q4 2023 vs. Q4 2022) despite R&D increases, demonstrating disciplined cost re-engineering.  
   - **Stock buybacks** ($70B+ authorizations) and **dividend growth** ($2.4B in 2024) boosted investor confidence.  

3. **Video Content Supremacy**:  
   - YouTube retained **50% CTV watch time in the U.S.** and **$50B+ ad/subscription revenue** in 2024, with Shorts monetization fueling the next growth wave.  

4. **Risks to Monitor**:  
   - FX volatility (-$136M headwind in Q2 2024), DOJ antitrust litigation, and **AI talent wars**.  
   - Supply-demand imbalances in Cloud capacity could affect 2025 revenue predictability.  

---

### **Management’s Long-Term Playbook**  
- **AI-First Product Roadmap**: Continue expanding **Gemini**, AI infrastructure (Trillium/Blackwell GPUs), and agent-based solutions (Astra).  
- **Cloud Sustainability**: Balance high Capex with **multi-cloud security (Wiz)** and **enterprise partnerships** (e.g., Citi, Oracle).  
- **Monetization Diversification**: Expand **First Position ad placements**, **YouTube Premium Lite**, and **Google One tiered subscriptions**.  

This narrative underscores the company’s shift from **AI exploration (2022)** to **infrastructure dominance (2024–2025)**, with monetization now translating to financial strength. The balance sheet and innovation pipeline position the business for **sustained growth in AI-driven cloud**, multimedia, and enterprise solutions.